<h1>문자를 읽을 수 있는 딥러닝</h1>

문자를 읽는 기술인 OCR(Optical Character Recognition, 광학 문자 인식)은 산업적으로 매우 유용한 기술이이다다. 딥러닝이 도입된 이후 OCR 기술이 획기적으로 개선되어 많은 산업 도메인에 유용하게 사용될 수 있는 가능성이 열리고 있다. 

### 실습목표
---

- OCR의 과정을 이해합니다.
- 문자인식 결과의 표현방식을 이해합니다.
- 파이썬을 통해 OCR을 사용할 수 있습니다.

### 학습 목차
---

- 들어가며
- 기계가 읽을 수 있나요?
- 어떤 과정으로 읽을까요?
- 딥러닝 문자인식의 시작
- 사진 속 문자 찾아내기 - detection
- 사진 속 문자 읽어내기 - recognition
- keras-ocr 써보기
- 테서랙트 써보기
- 프로젝트 : 다양한 OCR 모델 비교하기

### 준비물
---

$ mkdir -p ~/aiffel/EXPLORATION/18/ocr_python



## 기계가 읽을 수 있나요?

사람이 문자를 읽으려면 어떤 과정을 거칠까? 문자가 있다는 것을 1)인식하고, 인식한 문자를 2)해독하는 과정을 거치는 것이 일반적이다. 

기계가 문자를 읽는 과정도 이와 다르지 않다. 컴퓨터 비전에서 사용하는 용어로 위의 두 단계를 번역하자면 문자의 존재를 1)Detection 하고, 어떤 문자인지 판독하는 2)Recognition 으로 나눌 수 있다.

구글 OCR API를 이용하여 어플리케이션을 만들 수 있다.

### STEP 1) 구글의 파이썬 API 인터페이스 모듈을 아래와 설치합니다.

$ pip install --upgrade google-api-python-client

$ pip install google-cloud-vision

### STEP 2) Google Cloud Vision API 사용

서비스 계정 및 인증키를 생성. 브라우저에서 다운로드한 인증키는 my_google_api_key.json이라는 파일명으로 변경. (파일은 프로젝트 이름으로 시작하는 파일명으로 자동 저장됩니다.)

cloud jupyter를 열고 my_google_api_key.json 파일을 ~/aiffel/EXPLORATION/18/ocr_python 디렉토리에 업로드

### STEP 3) API 사용 테스트

API를 활용하는 코드는 아래와 같다. 사용할 이미지를 cloud jupyter 를 이용하여 업로드 후 경로를 지정

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
        
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
       print('\n"{}"'.format(text.description))

    vertices = (['({},{})'.format(vertex.x, vertex.y)
                 for vertex in text.bounding_poly.vertices])

    print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
# 로컬 환경에서는 다운받은 인증키 경로가 정확하게 지정되어 있어야 합니다. 
# 클라우드 환경에서는 무시해도 좋습니다
!ls -l $GOOGLE_APPLICATION_CREDENTIALS

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] =  os.getenv('HOME')+'/aiffel/EXPLORATION/18/ocr_python/my_google_api_key.json'

# 입력 이미지 경로를 지정해 주세요.
# (예시) path = os.getenv('HOME')+'/aiffel/ocr_python/test_image.png'
path = path = os.getenv('HOME')+'/aiffel/EXPLORATION/18/ocr_python/interstellar-imax-poster.jpeg'   

# 위에서 정의한 OCR API 이용 함수를 호출해 봅시다.
detect_text(path)

## 어떤 과정으로 읽을까요?

구글 API에서는 문자의 영역을 사각형으로 표현하고 우측에 Block과 Paragraph로 구분해서 인식 결과를 나타내고 있었습니다. 구글 API가 이미지에 박스를 친 다음 박스별 텍스트의 내용을 알려준 것처럼, 문자 모델은 보통 두 단계로 이뤄집니다.

먼저 입력받은 사진 속에서 문자의 위치를 찾아냅니다. 이 과정을 Text Detection(문자검출) 이라고 합니다. 찾은 문자 영역으로부터 문자를 읽어내는 것은 Text Recognition(문자인식) 입니다. 예를 들어 아래 카카오의 OCR 모델은 먼저 문자가 있는 영역의 정보 (coord, Text Recognition)를 찾아내고, 각 영역에서 문자를 인식하고 있습니다. 이렇게 문자 인식 모델은 Text Detection과 Recognition 두 과정을 통해서 사진 속의 문자를 읽을 수 있게 됩니다.

content img
[출처: https://brunch.co.kr/@kakao-it/318]
위 그림에서는 문자의 영역을 표현하는 방법으로 사각형의 네 꼭짓점 좌표를 알려 주는 방법을 제시하고 있습니다. 하지만 다른 도형이나 표현 방법을 사용할 수도 있을 것입니다. 이런 방법을 연구한 많은 논문들이 있습니다만, 아래에 그중 하나를 소개합니다.

